# Creation of Bayes Boundary Data (2/3):
This is meant to be run after the SignalDetection python script that creates the signal detections. This script creates a series of vectors to be stored in their own table which can be automatically plotted when queried. Uncomment code at bottom for plotting results.

### 1. Necessary Imports:

In [1]:
import numpy as np
import math 
from scipy import stats
import pandas as pd
import pymysql
import matplotlib as mpl
import matplotlib.pyplot as plt

from __future__ import division # Not neccessary in Python 3 and later
import scipy
from math import exp,sqrt

### 2) Functions to Estimate Probability Distributions

In [2]:
#helper function for calculating pdfs and x coordinates: 
def generate_normal_probabilities(data, adhd):
    #calculating moments:
    adhdMu = scipy.mean(data[adhd == 1])    
    adhdSD = scipy.std(data[adhd == 1])
    healthyMu = scipy.mean(data[adhd == 0])
    healthySD = scipy.std(data[adhd == 0])

    
    
    #calculating x  values for the pdfs
    minimum = np.amin((adhdMu-3*adhdSD, healthyMu - 3*healthySD))
    if minimum < 0:
        minimum = 0
    x = np.linspace(minimum, 
                    np.max((adhdMu + 3*adhdSD, healthyMu + 3*healthySD)),
                    200)
    
    adhdProbs = np.zeros(np.shape(x)[0])
    healthyProbs = np.zeros(np.shape(x)[0])
    
    #calculating pdfs for each x value
    for i in np.arange(np.shape(x)[0]):
        adhdProbs[i] = scipy.stats.norm.pdf(x[i], loc = adhdMu, scale = adhdSD)
        healthyProbs[i] = scipy.stats.norm.pdf(x[i], loc = healthyMu, scale = healthySD)
            
    return x, adhdProbs, healthyProbs


    

In [3]:
#hel
def get_probability_distributions(data):
    rows = np.shape(data)[0]
    cols = np.shape(data)[1]
    adhd = data[:, 0]

    data = data[:, 1:cols]

        
    #I should modularize this, but I'm tired and lazy
    xOE, adhdOEProbs, healthyOEProbs = generate_normal_probabilities(data[:, 0], adhd)
    xCE, adhdCEProbs, healthyCEProbs = generate_normal_probabilities(data[:, 1], adhd)
    xRTV, adhdRTCProbs, healthyRTVProbs = generate_normal_probabilities(data[:, 2], adhd)
    xDP, adhdDPProbs, healthyDPProbs = generate_normal_probabilities(data[:, 3], adhd)
    xB, adhdBProbs, healthyBProbs = generate_normal_probabilities(data[:, 4], adhd)

    return  np.transpose(np.array((xOE, adhdOEProbs, healthyOEProbs, 
                        xCE, adhdCEProbs, healthyCEProbs,
                        xRTV, adhdRTCProbs, healthyRTVProbs,
                        xDP, adhdDPProbs, healthyDPProbs,
                        xB, adhdBProbs, healthyBProbs)))
        
            

### 3. DB connecting, pull, & insert

In [4]:
#initializes connection info for database
def connect():
    return pymysql.connect(host = "rm-j6cluj4576jdi6n6oo.mysql.rds.aliyuncs.com",
                           database = 'rnd_test', 
                           user='cognitiveleap', 
                           password= 'QWE@123456')

In [5]:
#Gets pertinent data from database

def get_data(CaseIds, raw):
    #connect to db
    db = connect()
    
    # prepare a cursor object using cursor() method
    cursor = db.cursor()
    dataPatient = []
    dataSignal = []

    # Prepare SQL query to INSERT a record into the database.
    for i in CaseIds:
        if raw:
            sql = """SELECT TargetsRtVariability, OmissionErrors, CommissionErrors 
                     FROM cpt_output_results WHERE Block = 0 AND CasdId = """ + str(i) 
            cursor.execute(sql)
            # Fetch all the rows in a list of lists.
            results = np.asarray(cursor.fetchall())
            dataPatient.append(results)
            
            sql = """SELECT DPrime, Beta, ADHD FROM signal_detection WHERE Block = 0 AND CaseID = """ + str(i)
            cursor.execute(sql)
            # Fetch all the rows in a list of lists.
            results = np.asarray(cursor.fetchall())
            dataSignal.append(results)
            
    """        else:
            sql = "SELECT PathLen, TimeActive, NumRot, TotalDeg, CasdId FROM head_features WHERE CasdId = %s "% str(i)
            #print(sql)
            cursor.execute(sql)
            results = np.asarray(cursor.fetchall())
            if (len(results) > 0):
                data.append(results[0])
     

    except:
         print ("Error: unable to fetch data")"""

    db.close()
    dataPatient = np.concatenate(np.asarray(dataPatient))
    dataSignal = np.concatenate(np.asarray(dataSignal))
    return np.concatenate((dataPatient, dataSignal), axis = 1)


In [6]:
#Inserts created vectors into plotting table

def insert_bayes(data):
    db = connect()
    cursor = db.cursor()
    for i in range(len(data)):
        #weird syntax for readability
        sql = "INSERT INTO bayes_bound_prob_plot"
        sql += "(Id,"
        sql += " XCoordinateOmissionErrors,"
        sql += " HealthyOmissionErrorsProbability,"
        sql += " ADHDOmissionErrorsProbability,"
        sql += " XCoordinateComissionErrors,"
        sql += " HealthyCommissionErrorsProbability,"
        sql += " ADHDCommissionErrorsProbability,"
        sql += " XCoordinateTargetsRTVariability,"
        sql += " HealthyTargetsRTVariabilityProbability,"
        sql += " ADHDTargetsRTvariabilityProbability,"
        sql += " XCoordinateDPrime,"
        sql += " HealthyDPrimeProbability,"
        sql += " ADHDDPrimeProbability,"
        sql += " XCoordinateBeta,"
        sql += " HealthyBetaProbability,"
        sql += " ADHDBetaProbability)"
        sql += " VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)" % (np.int(i),
                                                                                             data[i, 0], 
                                                                                             data[i, 1], 
                                                                                             data[i, 2], 
                                                                                             data[i, 3], 
                                                                                             data[i, 4], 
                                                                                             data[i, 5],
                                                                                             data[i, 6],
                                                                                             data[i, 7],
                                                                                             data[i, 8],
                                                                                             data[i, 9],
                                                                                             data[i, 10],
                                                                                             data[i, 11],
                                                                                             data[i, 12],
                                                                                             data[i, 13],
                                                                                             data[i, 14]) 
        # Execute the SQL command
        cursor.execute(sql)
        # Commit your changes in the database
        db.commit()
    db.close()

### 4. Main Function:

In [7]:
#Main function does errythang

def main_bayes(caseIds):
    allPatients = get_data(caseIds, True) 
    correctOrderForBayes = np.array((5, 1, 2, 0, 3, 4))

    bayesData =  get_probability_distributions(allPatients[:, correctOrderForBayes])
    """
    #plotting code. Uncomment if you want to error check plots.
    plt.plot(bayesData[:, 0], bayesData[:, 1],'b') # plotting t,b separately 
    plt.plot(bayesData[:, 0], bayesData[:, 2],'g') # plotting t,c separately 
    plt.title("Omission Errors")
    plt.show()
    
    plt.plot(bayesData[:, 3], bayesData[:, 4],'b') # plotting t,b separately 
    plt.plot(bayesData[:, 3], bayesData[:, 5],'g') # plotting t,c separately 
    plt.title("Commission Errors")
    plt.show()
    
    plt.plot(bayesData[:, 6], bayesData[:, 7],'b') # plotting t,b separately 
    plt.plot(bayesData[:, 6], bayesData[:, 8],'g') # plotting t,c separately 
    plt.title("RTV")
    plt.show()
    
    plt.plot(bayesData[:, 9], bayesData[:, 10],'b') # plotting t,b separately 
    plt.plot(bayesData[:, 9], bayesData[:, 11],'g') # plotting t,c separately 
    plt.title("DPrime")
    plt.show()
    
    plt.plot(bayesData[:, 12], bayesData[:, 13],'b') # plotting t,b separately 
    plt.plot(bayesData[:, 12], bayesData[:, 14],'g') # plotting t,c separately 
    plt.title("Beta")
    plt.show()
    """
    
    #insert_bayes(bayesData)

    return bayesData


    

In [16]:
#call main
bayesData = main_bayes(np.arange(1, 21, 1))
    

In [18]:
bayesData[:, 0] = np.round(bayesData[:,0], 2)

for i in np.arange(np.shape(bayesData)[0]):
    if bayesData[i, 1] == 0 and bayesData[i, 2] == 0:
        prob = 1
    else:
        prob = np.round(bayesData[i, 1]/(bayesData[i, 1] + bayesData[i, 2]), 4)
    print("{xCoord: ", 
          bayesData[i, 0], 
          ", ADHD: ", 
          bayesData[i, 1],
          ", Healthy: ",
          bayesData[i, 2], 
          ", PADHD:",
          prob, 
          "},")



{xCoord:  0.0 , ADHD:  0.0241536959863 , Healthy:  0.0439667237541 , PADHD: 0.3546 },
{xCoord:  0.13 , ADHD:  0.0249164447669 , Healthy:  0.0459118311999 , PADHD: 0.3518 },
{xCoord:  0.27 , ADHD:  0.0256906429938 , Healthy:  0.0478873627251 , PADHD: 0.3492 },
{xCoord:  0.4 , ADHD:  0.026475873304 , Healthy:  0.0498899443849 , PADHD: 0.3467 },
{xCoord:  0.54 , ADHD:  0.0272716889523 , Healthy:  0.0519159630096 , PADHD: 0.3444 },
{xCoord:  0.67 , ADHD:  0.0280776138061 , Healthy:  0.0539615733173 , PADHD: 0.3422 },
{xCoord:  0.81 , ADHD:  0.0288931424116 , Healthy:  0.0560227066929 , PADHD: 0.3403 },
{xCoord:  0.94 , ADHD:  0.0297177401367 , Healthy:  0.0580950816388 , PADHD: 0.3384 },
{xCoord:  1.08 , ADHD:  0.0305508433898 , Healthy:  0.0601742158893 , PADHD: 0.3367 },
{xCoord:  1.21 , ADHD:  0.0313918599174 , Healthy:  0.0622554401663 , PADHD: 0.3352 },
{xCoord:  1.35 , ADHD:  0.0322401691818 , Healthy:  0.0643339135384 , PADHD: 0.3338 },
{xCoord:  1.48 , ADHD:  0.0330951228194 , Heal